# Code Bert thêm phần preprocessing, thay maxlength từ 512 sang max length 120 
# Thay thế các emoji thành text


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os 
os.chdir('/content/drive/MyDrive/vietnamesesentiment ')

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.0MB 20.0MB/s 
     |████████████████████████████████| 3.2MB 53.6MB/s 
     |████████████████████████████████| 890kB 46.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=2c81c16038e6aa02871a8e067f461d04aab7cb8d3518fe452481377219100656
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
emoticons = {':(':'tiêu cực', '☹':'tiêu cực', '❌':'tiêu cực', '👎':'tiêu cực', '👹':'tiêu cực', '💀':'tiêu cực', '🔥':'tiêu cực', '🤔':'tiêu cực', '😏':'tiêu cực', '😐':'tiêu cực', '😑':'tiêu cực', '😒':'tiêu cực', '😓':'tiêu cực', '😔':'tiêu cực', '😕':'tiêu cực', '😖':'tiêu cực',
                      '😞':'tiêu cực', '😟':'tiêu cực', '😠':'tiêu cực', '😡':'tiêu cực', '😢':'tiêu cực', '😣':'tiêu cực', '😤':'tiêu cực', '😥':'tiêu cực', '😧':'negative', '😨':'tiêu cực', '😩':'tiêu cực', '😪':'tiêu cực', '😫':'tiêu cực', '😭':'tiêu cực', '😰':'tiêu cực', '😱':'tiêu cực',
                      '😳':'tiêu cực', '😵':'tiêu cực', '😶':'tiêu cực', '😾':'tiêu cực', '🙁':'tiêu cực', '🙏':'tiêu cực', '🚫':'tiêu cực', '>:[':'tiêu cực', ':-(':'tiêu cực', ':(':'tiêu cực', ':-c':'tiêu cực', ':c':'tiêu cực', ':-<':'tiêu cực', ':っC':'tiêu cực', ':<':'tiêu cực',
                      ':-[':'tiêu cực', ':[':'tiêu cực', ':{':'tiêu cực','=))':'tích cực',';)':'tích cực', '^^':'tích cực', '<3':'tích cực', '☀':'tích cực', '☺':'tích cực', '♡':'tích cực', '♥':'tích cực', '✌':'tích cực', '✨':'tích cực', '❣':'tích cực', '❤':'tích cực', '🌝':'tích cực', '🌷':'tích cực', '🌸':'tích cực',
                      '🌺':'tích cực', '🌼':'tích cực', '🍓':'tích cực', '🎈':'tích cực', '🐅':'tích cực', '🐶':'tích cực', '🐾':'tích cực', '👉':'tích cực', '👌':'tích cực', '👍':'tích cực', '👏':'tích cực', '👻':'tích cực', '💃':'tích cực', '💄':'tích cực', '💋':'tích cực',
                      '💌':'tích cực', '💎':'tích cực', '💐':'tích cực', '💓':'tích cực', '💕':'tích cực', '💖':'tích cực', '💗':'tích cực', '💙':'tích cực', '💚':'tích cực', '💛':'tích cực', '💜':'tích cực', '💞':'tích cực', ':-)':'tích cực', ':)':'tích cực', ':D':'tích cực', ':o)':'tích cực',
                      ':]':'tích cực', ':3':'tích cực', ':c)':'tích cực', ':>':'tích cực', '=]':'tích cực', '8)':'tích cực','😂':'tích cực','🙂':'tích cực','🤣':'tích cực','😆':'tích cực','😰':'tiêu cực','🤣':'tích cực','😁':'tích cực','🙃':'tiêu cực','😀':'tích cực', '😭':'tiêu cực','😋':'tích cực','💪':'tích cực','😍':'tích cực', '😘':'tích cực', '😅':'tích cực','😏':'tiêu cực','😕':'tiêu cực','🤗':'tích cực', '😚':'tích cực', '✌':'tích cực', '😝':'tích cực', '👏':'tích cực', '😒':'tiêu cực', '🤨':'tiêu cực','❌':'tiêu cực', '😥':'tiêu cực', '😔':'tiêu cực', '😖':'tiêu cực', '😜':'tích cực','😢':'tiêu cực','😤':'tiêu cực', '😓':'tiêu cực','😩':'tiêu cực','😧':'tiêu cực','😻':'tích cực','😣':'tiêu cực','😎':'tích cực', '😙':'tích cực','💞':'tích cực','😵':'tiêu cực','😳':'tiêu cực','😯':'tiêu cực','💯':'tích cực','😨':'tiêu cực', '😟':'tiêu cực','👎':'tiêu cực','😠':'tiêu cực','😦':'tiêu cực','😫':'tiêu cực'}

In [ ]:
len(emoticons)

124

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences

from transformers import *
import numpy as np
import argparse
from tqdm import tqdm 
import random
import transformers
import pandas as pd
from collections import Counter
from transformers import get_linear_schedule_with_warmup
import sys
sys.argv=['']
del sys

In [ ]:
df = pd.read_csv('combine.csv')
X = df['sentence'].values
Y = df['label'].values
index = np.arange(X.shape[0])
np.random.shuffle(index)
X = X[index]
Y = Y[index]

In [ ]:
len(X)

33112

In [ ]:
X[0]

'Vietcombank ngân hàng lừa đảo chuyên giam tiền ko chịu chuyển cho khách hàng. Ngân hàng lừa đảo cần tẩy chay'

In [ ]:
X[2]

'Ngon , phục vụ tốt nhưng ăn ko thoải mái và nhiều bằng bên sumo hay hana . Mình đi đám 6 người , mỗi đứa trung bình 25Ok mà ăn ko no nê gì . Trong khi sumo chịu khó bỏ thêm 5Ok là tụi mình ăn hết\nMenu : <'

In [ ]:
X[3:8]

array(['Chủ nhật mình đi học nhóm với mấy đứa bạn , mình chọn quán tapi vì nơi đây gần trường . Lúc vào quán thì nhân viên có thái độ rất coi thường khách , cô ta hỏi là : " uống gì ? " . Và khi mình vào chỗ ngồi thì cô ta mang nước lên với gương mặt rất khó chịu . Mình thấy hầu hết khách , cô ta đều có thái độ tương tự không chỉ riêng mình .',
       '• Không gian của quán dạng là một hình chữ nhật , decor 2 mặt y chang nhau , đi một hồi chẳng biết nãy mình ngồi ở đâu .\n• Chỉ thích mỗi không gian trên cao , gió mát ở đây còn về chất lượng thì chỉ mức trung bình .\n• À , có cái khó hiểu , đây vừa ghi là HomyLand vừa ghi là Nhà Hàng Cafe Thượng Uyển , vậy chắc Thượng Uyển mới là tên chính xác của quán này !',
       'Không gian quán rộng rãi deco rất tây , bánh nhà làm cũng ngon nữa , recommend các bạn nên thử thêm cả các món đá say nhé . Giá cả dễ chịu phục vụ cả ăn trưa và ăn tối 😊\nOne of the hippest place in Tra Vinh I must say . Delicious homemade baked good and tasty frappuccino 

In [ ]:
X[8:13]

array(['Ngày 2-9 đi uống trà sữa ở quán quen , không biết một tuần tới quán bao nhiêu lần mà vẫn muốn tới\nTrà sữa ngon , vị đậm\nKhông gian đẹp\nCảm thấy hài lòng',
       'Bị kết mr good tea dman ... Đồ uống vừa ngon , vừa rẻ đã thế phục vụ còn nhiệt tình nữa chứ ... . Nhân viên thì khá thân thiện ... Thỉnh thoảng còn có nhạc acoustic nữa ... . Thích quán dman',
       '10k đĩa nên chật lượng tạm . Có thể k quay lại . Ngắn họn súc tích . Chỗ hợi chật ẩm .',
       'Cơm gà ăn dở tệ , chiều về bị tào tháo rượt quá trời , giá lại mắc nữa chứ .',
       'Mình đến đây vào tối thứ 4 13/08 . Chỗ ngồi khá đẹp . Mình uống socola nóng thì thấy 1/3 cốc là bọt . Sờ thân cốc rất nóng nhưng nước thì vô cùng nguội . Không biết có phải các bạn cho nước vào cốc rồi đem quay không . Socola uống hơi nhạt . Mình nghĩ đến đây chỉ nên uống cafe thôi . >   <'],
      dtype=object)

In [ ]:
X[200:205]

array(['Tụi mình gọi 4 set ăn .\nMì ramen ít , vị cũng bình thường , bánh không hơn là bao .\nCái tụi mình thắc mắc là sao trong hình để 4 ly matcha lại mang ra 4 bình trà rong biển . Lại kêu in nhầm .\nĐi 1 lần không muốn quay lại lần 2 .',
       'Gogi là địa điểm mình thường xuyên ghé tới luôn . Về đồ nướng mình không chê điểm nào . Nhưng hôm đến Trung Hoà ăn , gọi nước chanh và chanh leo thì cả 2 k có đá và quá ngọt . 2 chị em phải cho thêm bao nhiêu đá và nước vào để uống : (',
       'Tào phớ dâu tằm . cái tên nghe hót nhất quán mà vào chửng mấy ai gọi . may ra có mấy người thử lần đầu như mình . cái nước dâu tằm ăn thật dị khi ăn vs phớ @ @ . ko muốn ăn lại lần 2.Mấy món ăn chính thì tạm . gọi là ăn cho có . chẳng ngon cũng chẳng ko ngon . dưa su hào và kim chi ăn thêm thật thảm họa . ăn chua kiểu bị ỏng . hay gọi là khú : ( ( ( gê cổ .',
       'Bún bò ở đây ăn ngon . Nước dùng đậm đà hợp với khẩu vị của mình , thịt bò mềm chấm nước me của quán ngon cực luôn 👍 đặc biệt là rất n

In [ ]:
print(np.unique(Y))

[0 1 2]


In [ ]:
import numpy as np
# import unidecode
import pandas as pd
import string
import re
import os
import shutil
import pickle 

# Preprocessing

In [ ]:
def transform(x):
  return [s.lower() for s in x]
# def remove_tone(x):
#   return [unidecode.unidecode(s) for s in x] 
def remove_duplicate(x):
  result = []
  for s in x:
      s = re.sub(r'([a-z])\1+', lambda m: m.group(1), s, flags=re.IGNORECASE)
      s = re.sub(r'([a-z][a-z])\1+', lambda m: m.group(1), s, flags=re.IGNORECASE)
      result.append(s)
  return result

In [ ]:
X = transform(X)
X = remove_duplicate(X)

In [ ]:
len(X)

33112

In [ ]:
X[0]

'vietcombank ngân hàng lừa đảo chuyên giam tiền ko chịu chuyển cho khách hàng. ngân hàng lừa đảo cần tẩy chay'

In [ ]:
X[8:13]

['ngày 2-9 đi uống trà sữa ở quán quen , không biết một tuần tới quán bao nhiêu lần mà vẫn muốn tới\ntrà sữa ngon , vị đậm\nkhông gian đẹp\ncảm thấy hài lòng',
 'bị kết mr god tea dman ... đồ uống vừa ngon , vừa rẻ đã thế phục vụ còn nhiệt tình nữa chứ ... . nhân viên thì khá thân thiện ... thỉnh thoảng còn có nhạc acoustic nữa ... . thích quán dman',
 '10k đĩa nên chật lượng tạm . có thể k quay lại . ngắn họn súc tích . chỗ hợi chật ẩm .',
 'cơm gà ăn dở tệ , chiều về bị tào tháo rượt quá trời , giá lại mắc nữa chứ .',
 'mình đến đây vào tối thứ 4 13/08 . chỗ ngồi khá đẹp . mình uống socola nóng thì thấy 1/3 cốc là bọt . sờ thân cốc rất nóng nhưng nước thì vô cùng nguội . không biết có phải các bạn cho nước vào cốc rồi đem quay không . socola uống hơi nhạt . mình nghĩ đến đây chỉ nên uống cafe thôi . >   <']

In [ ]:
X[1]

'tổ chức sinh nhật con trai yêu tại đây mới cảm nhận được \xa0 dịch vụ của mai thành .\nkhông gian lý tưởng , món ăn tuyệt vời , đặc biệt nhân viên rất nhiệt tình .\ncác bạn xem nhé . \xa0\nnếu có dịp tụ họp nữa mình sẽ ghé mai thành ( ở đây gà ta hấp lá chanh cực ngon . . )'

In [ ]:
train_text = []
for text in X:
   for k, v in emoticons.items():
        text = text.replace(k, v)
   train_text.append(text)

In [ ]:
train_text[0]

'cơm xèo ở đây ngon nhưng lại rất rẻ , rẻ hơn nhiều so với per lunch . mình ăn ở per lunch môt phần gần 200 ngàn , nhưng ở đây môt phần cơm xèo chỉ dưới 100k , ăn cực kì ngon . ngoài ra còn có miến , mì ... đa dạng hơn . nên ghé thử các bạn ạ tích cực'

In [ ]:
train_text[200:205]

['tụi mình gọi 4 set ăn .\nmì ramen ít , vị cũng bình thường , bánh không hơn là bao .\ncái tụi mình thắc mắc là sao trong hình để 4 ly matcha lại mang ra 4 bình trà rong biển . lại kêu in nhầm .\nđi 1 lần không muốn quay lại lần 2 .',
 'gogi là địa điểm mình thường xuyên ghé tới luôn . về đồ nướng mình không chê điểm nào . nhưng hôm đến trung hoà ăn , gọi nước chanh và chanh leo thì cả 2 k có đá và quá ngọt . 2 chị em phải cho thêm bao nhiêu đá và nước vào để uống : (',
 'tào phớ dâu tằm . cái tên nghe hót nhất quán mà vào chửng mấy ai gọi . may ra có mấy người thử lần đầu như mình . cái nước dâu tằm ăn thật dị khi ăn vs phớ @ @ . ko muốn ăn lại lần 2.mấy món ăn chính thì tạm . gọi là ăn cho có . chẳng ngon cũng chẳng ko ngon . dưa su hào và kim chi ăn thêm thật thảm họa . ăn chua kiểu bị ỏng . hay gọi là khú : ( ( ( gê cổ .',
 'bún bò ở đây ăn ngon . nước dùng đậm đà hợp với khẩu vị của mình , thịt bò mềm chấm nước me của quán ngon cực luôn tích cực đặc biệt là rất nhiều thịt bò nhé 

In [ ]:
X = train_text

In [ ]:
X[0]

'vietcombank ngân hàng lừa đảo chuyên giam tiền ko chịu chuyển cho khách hàng. ngân hàng lừa đảo cần tẩy chay'

In [ ]:
X[8:13]

['ngày 2-9 đi uống trà sữa ở quán quen , không biết một tuần tới quán bao nhiêu lần mà vẫn muốn tới\ntrà sữa ngon , vị đậm\nkhông gian đẹp\ncảm thấy hài lòng',
 'bị kết mr god tea dman ... đồ uống vừa ngon , vừa rẻ đã thế phục vụ còn nhiệt tình nữa chứ ... . nhân viên thì khá thân thiện ... thỉnh thoảng còn có nhạc acoustic nữa ... . thích quán dman',
 '10k đĩa nên chật lượng tạm . có thể k quay lại . ngắn họn súc tích . chỗ hợi chật ẩm .',
 'cơm gà ăn dở tệ , chiều về bị tào tháo rượt quá trời , giá lại mắc nữa chứ .',
 'mình đến đây vào tối thứ 4 13/08 . chỗ ngồi khá đẹp . mình uống socola nóng thì thấy 1/3 cốc là bọt . sờ thân cốc rất nóng nhưng nước thì vô cùng nguội . không biết có phải các bạn cho nước vào cốc rồi đem quay không . socola uống hơi nhạt . mình nghĩ đến đây chỉ nên uống cafe thôi . >   <']

# Model 


In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--epochs', type=int, default=10)
parser.add_argument('--batch_size', type=int, default=8)
parser.add_argument('--maxlen', type=int, default=120)
parser.add_argument('--seed', type=int, default=42)
parser.add_argument('--lr', type=float, default=2e-5)
args = parser.parse_args()

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)

MAX_LEN = args.maxlen

In [ ]:
MAX_LEN

120

In [ ]:
def get_dataloader(sentences, labels, tokenizer, batch_size):
    input_ids = []
    filtered_labels = []
    for i, sent in tqdm(enumerate(sentences)):
        encoded_sent = tokenizer.encode(sent,add_special_tokens = True)
        if len(encoded_sent) <= 512:
            input_ids.append(encoded_sent)
            filtered_labels.append(labels[i])
    print(Counter(filtered_labels))
    inputs = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                                    value=0, truncating="post", padding="post")                                                     
    masks = []

    for sent in inputs:
        att_mask = [int(token_id > 0) for token_id in sent]
        masks.append(att_mask)

    inputs = torch.tensor(inputs)
    labels = torch.tensor(filtered_labels)
    masks = torch.tensor(masks)

    data = TensorDataset(inputs, masks, labels)
    sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)

    return dataloader

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
def get_baseline_acc(model, validation_dataloader, device):
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():        
            outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        logits = outputs[1]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    eval_accuracy = eval_accuracy/nb_eval_steps
    return eval_accuracy

In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", num_labels=3)
model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
X_train = X[:-10000]
Y_train = Y[:-10000]
X_test = X[-10000:]
Y_test = Y[-10000:]
print(Counter(Y_train), Counter(Y_test))
train_dataloader = get_dataloader(X_train, Y_train, tokenizer, args.batch_size)
test_dataloader = get_dataloader(X_test, Y_test, tokenizer, args.batch_size)

optimizer = AdamW(model.parameters(),lr = args.lr)
total_steps = len(train_dataloader) * args.epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = int(0.1*total_steps),
                                            #warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
                                            #t_total = total_steps)

92it [00:00, 914.87it/s]

Counter({2: 11701, 0: 11061, 1: 350}) Counter({2: 5121, 0: 4714, 1: 165})


4839it [00:05, 965.46it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (645 > 512). Running this sequence through the model will result in indexing errors
23112it [00:23, 966.77it/s]


Counter({2: 11701, 0: 11059, 1: 350})


10000it [00:10, 981.50it/s]


Counter({2: 5121, 0: 4714, 1: 165})


In [ ]:
len(X_train)

23112

In [ ]:
len(X_test)

10000

In [ ]:
for epoch_i in range(args.epochs):
    total_loss = 0
    model.train()
        
    # For each batch of training data...
    for step, batch in tqdm(enumerate(train_dataloader)):
        model.train()
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
                
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        loss = outputs[0]
        loss.backward()

        total_loss += loss.item()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        model.zero_grad()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))

    test_accuracy = get_baseline_acc(model, test_dataloader, device)
    print(" Test acc {}".format( test_accuracy))

print("")
print("Training complete!")

2889it [11:16,  4.27it/s]



  Average training loss: 0.54


0it [00:00, ?it/s]

 Test acc 0.8443


2889it [11:16,  4.27it/s]


...


  Average training loss: 0.05


0it [00:00, ?it/s]

 Test acc 0.8611


2889it [10:59,  4.38it/s]



  Average training loss: 0.03
 Test acc 0.8615

Training complete!


In [ ]:
# chạy lên đc 90%